In [ ]:
!pip install gensim==3.8.3
!pip install unidecode
!pip install torch
!pip install sentencepiece
!pip install transformers

In [2]:
import time
import pickle
from pathlib import Path
import gensim
import pandas as pd
import numpy as np
import sklearn
import re
from unidecode import unidecode
import functools
import operator
import psutil
from joblib import Parallel,delayed
from functools import partial
from time import time
import torch
import torch.nn as nn
from tqdm import tqdm
import os
os.chdir("C:\\Users\\theo.roudil-valentin\\Documents\\Resume\\MLSUM")

In [4]:
base=pickle.load(open('MLSUM_fr_val.pickle','rb'))
pd.DataFrame(base.text.values[:10]).to_csv('MLSUM_fr_val.csv',sep=';')

In [4]:
from fats import Make_Extractive,Word_Cleaning
W2V=None
print("CPU dispo :",psutil.cpu_count())
cpu_max=24

CPU dispo : 56


In [ ]:
fichiers=[i for i in os.listdir() if (i[-6:]=='pickle') and (i[:5]=='MLSUM')]
fichiers_val=[i for i in fichiers if 'test' in i]

for f in fichiers_val:
    print(f)
    base=pickle.load(open(f,'rb'))
    name=f.split('.')[0].split('fr')[-1][1:]

    print('Début du nettoyage du fichier',name,':')
    cleaning_time=time()
    WC=Word_Cleaning(n_jobs=cpu_max,sentence=True,threshold=True,seuil=2,lemma=False,seuil_carac=3)

    summary=base.summary.values
    summary=WC.make_summary(summary)
    pickle.dump(summary,open('summary_clean_'+name+'.pickle','wb'))

    text=base.text.values
    text=WC.make_documents(text)
    cleaning_end=time()
    pickle.dump(text,open('text_clean_'+name+'.pickle','wb'))
    print("Durée du nettoyage :",round((cleaning_end-cleaning_time)/60,2),"minutes.")


In [28]:
summaries=[i for i in os.listdir() if 'summary' in i]
texts=[i for i in os.listdir() if 'text' in i]

text=pickle.load(open(texts[1],'rb'))
summary=pickle.load(open(summaries[1],'rb'))
longueur=[len(i) for i in sentence]

assert len(text)==len(summary)

dim=100
fenetre=20
minimum=1
epo=5

ME=Make_Extractive(cpu=cpu_max)

sentence=ME.make_w2v_sentences(text)

In [29]:
#Pour 15800, le W2V a pris 3,3 minutes 
if W2V==None:
    import gensim
    try:
        W2V=gensim.models.Word2Vec(size=dim,window=fenetre,min_count=minimum)
    except:
        W2V=gensim.models.Word2Vec(vector_size=dim,window=fenetre,min_count=minimum)
    W2V.build_vocab(sentence)
    print("Démarrage de l'entraînement du modèle Word2Vec.")
    start=time()
    W2V.train(sentence,total_examples=W2V.corpus_count,epochs=epo)
    end=time()
    print("Le modèle W2V est désormais entraîné et cela a pris :",round((end-start)/60,2),"minutes.")

#Pour 15800, le dico vocab a pris 0 minutes 
start=time()
try:
    vocab=list(W2V.wv.vocab.keys())
except:
    vocab=list(set(W2V.wv.key_to_index))
end=time()
print("La création du dico vocab a pris:",round((end-start)/60,2),"minutes.")

#Pour 15800, le découpage a pris 0 minutes 
start=time()
text=Parallel(n_jobs=cpu_max)(delayed(ME.make_splitting)(s) for s in text)
#text=[[i.split() for i in s] for s in docs]

summary=ME.make_splitting(summary)

#summary=[[i for i in s.split() if i in vocab] for s in summary]
end=time()
print("Le découpage des phrases a pris:",round((end-start)/60,2),"minutes.")


La création du dico vocab a pris: 0.0 minutes.
Le découpage des phrases a pris: 0.09 minutes.


In [36]:
sent=text[0]
s=sent[0]
torch.stack(
             [cosim(torch.as_tensor(W2V.wv[i]),torch.as_tensor(W2V.wv[summary[text.index(sent)]]))
             for i in s]).mean()

tensor(0.0067)

In [40]:
gensim.__version__>'4.0.0'

True

In [ ]:
cosim=torch.nn.CosineSimilarity(-1)

score=[]
erreur=[]
for sent in tqdm(text):
    score_=[]
    for s in sent:
        try:
            score_.append(torch.stack(
             [cosim(torch.as_tensor(W2V[i]),torch.as_tensor(W2V[summary[text.index(sent)]]))
             for i in s]).mean())
        except:
            erreur.append([text.index(sent),sent.index(s)])
            print("Attention, l'élément",erreur[-1],"n'a pas pu être encodé.")
            continue
    try:
        score.append(torch.stack(score_))
    except:
        score.append(torch.Tensor())

In [13]:
from fats import Make_Extractive

summaries=[i for i in os.listdir() if ('summary_word_3' in i) and ('train' not in i)]
texts=[i for i in os.listdir() if ('text_word_3' in i) and ('train' not in i)]
W2V=pickle.load(open('W2V_train.pickle','rb'))

cpu_max=30
n=10


In [14]:
for summary,text in zip(summaries,texts):
        name='train_'+summary.split('.')[0].split('_')[-1]
        print(name)
        text=pickle.load(open(text,'rb'))
        summary=pickle.load(open(summary,'rb'))
        score_final=[]
        erreur_f=[]
        for l in range(n):
            if l<3:
                l_1=int(len(text)*(l)/n)
                l_2=int(len(text)*(l+1)/n)
                print(l_1,l_2)
                text_=text[l_1:l_2]
                summary_=summary[l_1:l_2]
                print("Début de la création de l'output :")
                output_time=time()
                ME=Make_Extractive(cpu=cpu_max,fenetre=20,minimum=1,d=100,epochs=25)
                score,text_2,summary_2,erreur=ME.make_output(text_,summary_,W2V=W2V)
                output_end=time()
                print("Durée de la création d'output :",round((output_end-output_time)/60,2),"minutes.") #42 minutes pour 15,8K lignes
                print("Nombre d'erreurs pendant l'encodage du test :",round(len(erreur)/len(text_)*100,2),"%")
                score_final+=score
                erreur_f+=erreur
                pickle.dump(score,open('score_'+name+'_'+str(l+1)+'.pickle','wb'))
            else:
                break
        score=pickle.load(open('score_train_3.pickle','rb'))
        erreur=pickle.load(open('erreur_word_train_3.pickle','rb'))
        score_final=score+score_final
        pickle.dump(score_final,open('score_'+name+'.pickle','wb'))
        erreur_f=erreur_f+erreur
        pickle.dump(erreur_f,open('erreur_word_'+name+'.pickle','wb'))
        for l in range(n):
            os.remove('score_'+name+'_'+str(l+1)+'.pickle')
        #print(score_final)
        #pickle.dump(W2V,open('W2V_'+name+'.pickle','wb'))
        #pickle.dump(text_2,open('text_word_'+name+'.pickle','wb'))
        #pickle.dump(summary_2,open('summary_word_'+name+'.pickle','wb'))


train_3


  0%|          | 3/9822 [00:00<06:18, 25.97it/s]

0 9822
Début de la création de l'output :


100%|██████████| 9822/9822 [12:11<00:00, 13.42it/s]


Durée de la création d'output : 12.2 minutes.
Nombre d'erreurs pendant l'encodage du test : 5.06 %


...

100%|██████████| 9823/9823 [12:56<00:00, 12.65it/s]


Durée de la création d'output : 12.94 minutes.
Nombre d'erreurs pendant l'encodage du test : 4.72 %


  0%|          | 3/9822 [00:00<05:29, 29.76it/s]

19645 29467
Début de la création de l'output :


 62%|██████▏   | 6107/9822 [07:06<06:30,  9.52it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
len(score_final)

29467

In [21]:
score=pickle.load(open('score_train_3.pickle','rb'))
print(len(score))
erreur=pickle.load(open('erreur_word_train_3.pickle','rb'))
print(len(erreur))
score_final=score+score_final
print(len(score_final))

68758
3477
98225


In [22]:
pickle.dump(score_final,open('score_'+name+'.pickle','wb'))
erreur_f=erreur_f+erreur
print(len(erreur_f))
pickle.dump(erreur_f,open('erreur_word_'+name+'.pickle','wb'))

4827


In [ ]:
pickle.dump(score_final,open('score_train_1_1.pickle','wb'))
pickle.dump(erreur_f,open('erreur_word_train_1_1.pickle','wb'))

In [12]:
len(score)+29467,len(text)

(98225, 98225)

In [23]:
#text=pickle.load(open('text_word_3.pickle','rb'))

x=0
for t in text:
    x+=len(t)
print("Nombre de données :",x)
#erreur=pickle.load(open('erreur_word_train_3.pickle','rb'))
print("Nombre d'erreurs pendant l'encodage du text :",round(len(erreur)/x*100,2),"%")

Nombre de données : 2242804
Nombre d'erreurs pendant l'encodage du text : 0.16 %


In [30]:
len(summary[0])

19

In [24]:
text=pickle.load(open('text_clean_train_3.pickle','rb'))
score=pickle.load(open('score_train_3.pickle','rb'))
print(len(text),len(score))

relou=[i for i in range(len(score)) if len(text[i])!=len(score[i])]
print(len(relou))

def remove_empty(text):
    while '' in text:
          text.remove('')
    return text

text_ne=Parallel(n_jobs=cpu_max)(delayed(remove_empty)(t) for t in text)
print(len(text_ne),len(score))
relou=[i for i in range(len(score)) if len(text_ne[i])!=len(score[i])]
print(len(relou))

index=[i for i in range(len(text_ne)) if i not in relou]

text_ne=[text_ne[i] for i in index]
score_ne=[score[i] for i in index]
print(len(text_ne),len(score_ne))

relou=[i for i in range(len(text_ne)) if len(text_ne[i])!=len(score_ne[i])]
print(len(relou))

98225 98225
95722
98225 98225
95709
2516 2516
0


In [ ]:
W2V=pickle.load(open('W2V_train.pickle','rb'))

erreur_text={}
score_relou=[]
for k in relou:
    nul=[]
    for i in range(len(text[k])):
        try:
            W2V.wv[text[k][i].split()]
        except:
            #print(k,i)
            nul.append(i)
    erreur_text[k]=nul
    if len(erreur_text[k])>0:
        for e in list(reversed(erreur_text[1915])):
            score_=torch.cat([score[k][:e],torch.Tensor([0]),score[k][e:]])
        score_relou.append(score_)

erreur_text

In [ ]:
#texts=[i for i in os.listdir() if 'text_clean' in i]
#scores=[i for i in os.listdir() if 'score' in i]

#for text,score in zip(texts,scores):
#name=text.split('.')[0].split('_')[-1]

#text=pickle.load(open(text,'rb'))
#score=pickle.load(open(score,'rb'))

print("Début de la création de l'encoding")
encoding_time=time()
ME=Make_Extractive(cpu_max)
dico=ME.make_encoding(text_ne,score_ne,voc_size=12000,split=1)
encoding_end=time()

In [ ]:
print("Durée de la création d'output :",round((encoding_end-encoding_time)/60,2),"minutes.")

pickle.dump(dico,open('dico_train_3.pickle','wb'))
print("Les données du val ont été sauvegardées !")

In [4]:
from transformers import CamembertTokenizer
tokenizer=CamembertTokenizer('essai.model')
ME=Make_Extractive(cpu=cpu_max)
doc_encod=partial(ME.document_encoding,tokenizer=tokenizer,dim=512)   
output=[]#Parallel(n_jobs=cpu_max)(delayed(
erreur=[]
for t,s in tqdm(zip(text_ne,score_ne)):
    try:
        output.append(doc_encod(t,s))
    except:
        erreur.append(text_ne.index(t))

15128it [02:13, 113.61it/s]


In [5]:
erreur

[9102, 9151]

In [8]:
len(score_ne[9102]),len(text_ne[9102])

(4, 4)

In [7]:
print('ouais')

ouais
